# Lab 3: Determination of Protein Concentration

In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib as mp
import matplotlib.pyplot as plt 
from scipy import stats

from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

In [2]:
BR = pd.read_csv('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 3 - Determination of Protein Concentrations/Data/Biuret.csv', 
                 header=1)

LR = pd.read_csv('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 3 - Determination of Protein Concentrations/Data/Lowry.csv', 
                 header=1)

BF = pd.read_csv('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 3 - Determination of Protein Concentrations/Data/Bradford.csv', 
                 header=1)

UN = pd.read_csv('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 3 - Determination of Protein Concentrations/Data/Unknown.csv', 
                 header=1)

In [3]:
# Get average absorbance
BR_ave = pd.DataFrame(BR.iloc[:,1:].mean(axis=1))
LR_ave = pd.DataFrame(LR.iloc[:,1:].mean(axis=1))
BF_ave = pd.DataFrame(BF.iloc[:,1:].mean(axis=1))
UN_ave = pd.DataFrame(UN.iloc[:,1:].mean(axis=1))
BR_ave.columns = ['Average']
LR_ave.columns = ['Average']
BF_ave.columns = ['Average']
UN_ave.columns = ['Average']
BR = BR.join(BR_ave)
LR = LR.join(LR_ave)
BF = BF.join(BF_ave)
UN = UN.join(UN_ave)

In [4]:
# Get mg of BSA 
BR_g = pd.DataFrame(BR['BSA (10g/l) [ml]'] * 10)
LR_g = pd.DataFrame(LR['BSA (100 mg/l) [ml]'] * 0.1)
BF_g = pd.DataFrame(BF['BSA (100 mg/l) [ul]'] * 10**(-4))
BR_g.columns = ['BSA [mg]']
LR_g.columns = ['BSA [mg]']
BF_g.columns = ['BSA [mg]']
BR = BR.join(BR_g)
LR = LR.join(LR_g)
BF = BF.join(BF_g)

In [5]:
BR

,BSA (10g/l) [ml],Set 1,Set 2,Average,BSA [mg]
0,0.0,0.000,0.000,0.0000,0.0
1,0.2,0.045,0.079,0.0620,2.0
2,0.4,0.109,0.160,0.1345,4.0
3,0.6,0.194,NaN,0.1940,6.0
4,0.8,NaN,0.337,0.3370,8.0
5,1.0,0.361,0.372,0.3665,10.0


In [6]:
LR

,BSA (100 mg/l) [ml],Set 1,Set 2,Average,BSA [mg]
0,0.0,0.000,0.000,0.0000,0.00
1,0.2,0.133,0.118,0.1255,0.02
2,0.4,0.231,0.237,0.2340,0.04
3,0.6,0.347,0.329,0.3380,0.06
4,0.8,0.389,0.444,0.4165,0.08
5,1.0,0.520,0.523,0.5215,0.10


In [7]:
BF

,BSA (100 mg/l) [ul],Set 1,Set 2,Average,BSA [mg]
0,0,-0.005,0.005,0.0000,0.000
1,20,0.081,0.098,0.0895,0.002
2,40,0.144,0.146,0.1450,0.004
3,60,0.166,0.160,0.1630,0.006
4,80,0.205,0.198,0.2015,0.008
5,100,0.270,0.271,0.2705,0.010


In [8]:
UN

,Assay,Set 1,Set 2,Average
0,Biuret 550nm,0.366,0.390,0.3780
1,Lowry 750 nm,0.489,0.488,0.4885
2,Bradford 595nm,0.231,0.238,0.2345


## F. Practical Assessment

In [13]:
def plot_abs(Dat, X, Y, Abs, Yerr, ploterror=False):
    slope, intercept, r_value, p_value, std_err = stats.linregress(Dat[X], Dat[Y])

    plt.figure()
    sns.regplot(x = X, y=Y, data=Dat, 
                scatter_kws={'s':8}, 
                line_kws={'linewidth':2, 'label':r"A={0:.3g}M+({1:.3g}); R$^2$={2:.3g}".format(slope, intercept, r_value**2)})
    
    if ploterror == True: 
        plt.errorbar(x=Dat[X], y=Dat[Y], yerr=Yerr, scatterpoints=None, linewidth=0, elinewidth=1, capsize=2,
                    label='Uncertainty: {0:.4f}'.format(Yerr))
    
    F_title = r'Changes in average absorbance at {0} nm ($\bar{{A}}_{1}$) with respect to the quantity of BSA, with a 95% confidence interval'.format(Abs, {Abs})
    plt.title("\n".join(wrap(F_title, 80)))
    plt.ylabel(r'$\bar{{A}}_{0}$'.format({Abs}))
    plt.legend(loc=2)
    plt.grid()
    plt.show()

In [14]:
plot_abs(Dat=BR, X='BSA [mg]', Y='Average', Abs=550, Yerr=(2*0.0005**2)**0.5)
plot_abs(Dat=LR, X='BSA [mg]', Y='Average', Abs=750, Yerr=(2*0.0005**2)**0.5)
plot_abs(Dat=BF, X='BSA [mg]', Y='Average', Abs=595, Yerr=(2*0.0005**2)**0.5)

<Figure size 432x288 with 1 Axes>

<Figure size 432x288 with 1 Axes>

<Figure size 432x288 with 1 Axes>